In [ ]:
# make sure to use the latest version of the openai python package
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "sk-proj-x9XrkmMcchsC5r314ufCT3BlbkFJ0SXyeSsSNF7xYH0fiHvn"))

In [ ]:
# Read in the dataset we'll use for this task.
# This will be the RecipesNLG dataset, which we've cleaned to only contain documents from www.cookbooks.com
#recipe_df = pd.read_csv("data/cookbook_recipes_nlg_10k.csv")
drugs_df = pd.read_csv("/content/drugs_side_effects_AI.csv")
drugs_df.head()

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,Vyvanse,ADHD,hives; difficult breathing; swelling of your f...,lisdexamfetamine,CNS stimulants,NaN,100%,Rx,C,2,X,NaN,ADHD (Attention Deficit Hyperactivity Disorder...,7.3,768.0,https://www.drugs.com/vyvanse.html,https://www.drugs.com/condition/attention-defi...
1,amphetamine,ADHD,hives ; difficult breathing; swelling of your ...,amphetamine,CNS stimulants,"Adzenys ER, Adzenys XR-ODT, Dyanavel XR, Eveke...",7%,Rx,C,2,X,Adderall: https://www.drugs.com/adderall.html ...,ADHD (Attention Deficit Hyperactivity Disorder...,6.4,119.0,https://www.drugs.com/amphetamine.html,https://www.drugs.com/condition/attention-defi...
2,Ritalin LA,ADHD,"Along with its needed effects, a medicine may ...",methylphenidate (oral route),CNS stimulants,NaN,7%,Rx,C,2,X,Adderall: https://www.drugs.com/adderall.html ...,ADHD (Attention Deficit Hyperactivity Disorder...,7.9,17.0,https://www.drugs.com/cons/ritalin-la.html,https://www.drugs.com/condition/attention-defi...
3,Kapvay,ADHD,hives ; difficult breathing; swelling of your ...,clonidine (oral),"Antiadrenergic agents, centrally acting",NaN,6%,Rx,C,N,X,Adderall: https://www.drugs.com/adderall.html ...,ADHD (Attention Deficit Hyperactivity Disorder...,7.4,18.0,https://www.drugs.com/kapvay.html,https://www.drugs.com/condition/attention-defi...
4,Methylin,ADHD,"Along with its needed effects, a medicine may ...",methylphenidate (oral route),CNS stimulants,NaN,6%,Rx,C,2,X,Adderall: https://www.drugs.com/adderall.html ...,ADHD (Attention Deficit Hyperactivity Disorder...,8.2,14.0,https://www.drugs.com/cons/methylin.html,https://www.drugs.com/condition/attention-defi...


In [ ]:
training_data = []
system_message="You are a helpful pharmaceutical assistant. Provide drug recommendations based on detailed medical condition descriptions when requested"
def create_user_message(condition_description):
    return f"I have symptoms of {condition_description[:150]}... What medication can help?"

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row['medical_condition_description'])
    messages.append({"role": "user", "content": user_message})

    assistant_message = f"For the symptoms and condition described, {row['drug_name']} is commonly recommended. It treats {row['medical_condition']}."
    messages.append({"role": "assistant", "content": assistant_message})

    return {"messages": messages}


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming 'drugs_df' is your DataFrame containing the data

# Shuffle the DataFrame
drugs_df = drugs_df.sample(frac=1).reset_index(drop=True)

# Calculate the number of rows for train, validation, and test sets
total_rows = len(drugs_df)
train_frac = 0.70
val_frac = 0.15
test_frac = 0.15  # This can be adjusted if the sum doesn't add up to 1.0 exactly due to rounding

# Calculate split indices
train_end = int(train_frac * total_rows)
val_end = train_end + int(val_frac * total_rows)

# Slice the DataFrame
train_df = drugs_df[:train_end]
val_df = drugs_df[train_end:val_end]
test_df = drugs_df[val_end:]

# Apply the 'prepare_example_conversation' function to each subset
training_data = train_df.apply(prepare_example_conversation, axis=1).tolist()
validation_data = val_df.apply(prepare_example_conversation, axis=1).tolist()
test_data = test_df.apply(prepare_example_conversation, axis=1).tolist()

# Example output to verify
# Print the first 5 examples from the training data to check
for example in training_data[:5]:
    print(example)


{'messages': [{'role': 'system', 'content': 'You are a helpful pharmaceutical assistant. Provide drug recommendations based on detailed medical condition descriptions when requested'}, {'role': 'user', 'content': 'I have symptoms of Asthma Other names: Bronchial Asthma; Exercise-induced asthma; Wheezing Asthma is an inflammatory disorder of the airways, characterized by periodic a... What medication can help?'}, {'role': 'assistant', 'content': 'For the symptoms and condition described, Dulera is commonly recommended. It treats Asthma.'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful pharmaceutical assistant. Provide drug recommendations based on detailed medical condition descriptions when requested'}, {'role': 'user', 'content': 'I have symptoms of Urinary Tract Infection Other names: Acute Bacterial Cystitis; Catheter-Associated Urinary Tract Infection; Chronic Urinary Tract Infection; Cystitis,... What medication can help?'}, {'role': 'assistant', 'content': 'For 

In [ ]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [ ]:
training_file_name = "tmp_recipe_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_recipe_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [ ]:
# print the first 5 lines of the training file
!head -n 5 tmp_recipe_finetune_training.jsonl

{"messages": [{"role": "system", "content": "You are a helpful pharmaceutical assistant. Provide drug recommendations based on detailed medical condition descriptions when requested"}, {"role": "user", "content": "I have symptoms of Asthma Other names: Bronchial Asthma; Exercise-induced asthma; Wheezing Asthma is an inflammatory disorder of the airways, characterized by periodic a... What medication can help?"}, {"role": "assistant", "content": "For the symptoms and condition described, Dulera is commonly recommended. It treats Asthma."}]}
{"messages": [{"role": "system", "content": "You are a helpful pharmaceutical assistant. Provide drug recommendations based on detailed medical condition descriptions when requested"}, {"role": "user", "content": "I have symptoms of Urinary Tract Infection Other names: Acute Bacterial Cystitis; Catheter-Associated Urinary Tract Infection; Chronic Urinary Tract Infection; Cystitis,... What medication can help?"}, {"role": "assistant", "content": "For 

In [ ]:
with open(training_file_name, "rb") as training_fd:
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(validation_file_name, "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-lQygf0qmvpLpDcvlVFP4G9cj
Validation file ID: file-nKqifWokX45fZj851RgIGHjx


In [ ]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="recipe-ner",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-Juoo2dAVYXIpt9FNV3q6Zzrk
Status: validating_files


In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)


Job ID: ftjob-Juoo2dAVYXIpt9FNV3q6Zzrk
Status: succeeded
Trained Tokens: 344334


In [ ]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 1754/1769: training loss=0.24
Step 1755/1769: training loss=0.31
Step 1756/1769: training loss=0.24
Step 1757/1769: training loss=0.19
Step 1758/1769: training loss=0.22
Step 1759/1769: training loss=0.14
Step 1760/1769: training loss=0.24
Step 1761/1769: training loss=0.24
Step 1762/1769: training loss=0.17
Step 1763/1769: training loss=0.12
Step 1764/1769: training loss=0.28
Step 1765/1769: training loss=0.17
Step 1766/1769: training loss=0.26
Step 1767/1769: training loss=0.16, full validation loss=0.30
Step 1768/1769: training loss=0.32
Step 1769/1769: training loss=0.31
Checkpoint created at step 1178 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:recipe-ner:9Npbk7N9:ckpt-step-1178
Checkpoint created at step 1767 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:recipe-ner:9NpblSNg:ckpt-step-1767
New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal:recipe-ner:9Npbl2pO
The job has successfully completed


In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:recipe-ner:9Npbl2pO


In [ ]:
test_df = drugs_df.loc[201:300]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You are a helpful pharmaceutical assistant. Provide drug '
             'recommendations based on detailed medical condition descriptions '
             'when requested',
  'role': 'system'},
 {'content': 'I have symptoms of '
             'drug_name                                                                '
             'ritonavir\n'
             'medical_condition                                                         '
             'AIDS/HIV\n'
             'side_effects                     irregular heartbeats, or a '
             'light-headed feelin...\n'
             'generic_name                                                             '
             'ritonavir\n'
             'drug_classes                               Antiviral boosters, '
             'Protease inhibitors\n'
             'brand_names                                                                 '
             'Norvir\n'
             'activity                                         

In [ ]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

For the symptoms and condition described, ritonavir is commonly recommended. It treats AIDS/HIV.
